In [147]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
from sklearn.metrics import make_scorer
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, cross_val_score, train_test_split
from sklearn.metrics import mean_absolute_error
import scipy 
from scipy.stats import linregress
from sklearn.linear_model import SGDRegressor, Ridge

In [2]:
dataset = pd.read_excel("dataset_renamed.xlsx")

<h1>Data review

In [3]:
dataset.describe()

,id,target,Расстояние до метро,"Суммарное кол-во отзывов в категории ""АЗС"" в радиусе 150м","Суммарное кол-во отзывов в категории ""Алкомаркеты"" в радиусе 150м","Суммарное кол-во отзывов в категории ""Аптеки"" в радиусе 150м","Суммарное кол-во отзывов в категории ""Банки"" в радиусе 150м","Суммарное кол-во отзывов в категории ""Бизнес_центры"" в радиусе 150м","Суммарное кол-во отзывов в категории ""ВУЗы"" в радиусе 150м","Суммарное кол-во отзывов в категории ""Вокзалы"" в радиусе 150м",...,Суммарный вечерний трафик сигналов рабочих в радиусе 140м,Суммарный трафик сигналов рабочих в радиусе 140м,Суммарный утренний трафик юзеров рабочих в радиусе 140м,Суммарный дневной трафик юзеров рабочих в радиусе 140м,Суммарный вечерний трафик юзеров рабочих в радиусе 140м,Суммарный трафик юзеров рабочих в радиусе 140м,Тип БЦ,Тип хаб,Тип улица,Тип ТЦ
count,98.000000,88.000000,98.000000,98.000000,98.000000,98.000000,98.000000,98.000000,98.000000,98.000000,...,98.000000,98.000000,98.000000,98.000000,98.000000,98.000000,98.000000,98.000000,98.000000,98.000000
mean,74.183673,1.374264,8155.419924,26.285714,114.591837,209.071429,260.500000,1879.000000,65.979592,1921.081633,...,119.336735,616.908163,5.632653,6.724490,5.091837,9.183673,0.030612,0.071429,0.489796,0.408163
std,42.133403,0.182284,7434.223789,179.532669,256.443954,224.739127,332.486299,17983.545644,321.662466,13721.481794,...,140.165436,615.651111,4.491430,4.748913,4.096986,6.216617,0.173150,0.258863,0.502466,0.494021
min,0.000000,1.000000,485.615204,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,37.250000,1.269808,2737.660223,0.000000,0.000000,56.250000,0.000000,0.000000,0.000000,0.000000,...,18.250000,144.000000,3.000000,4.000000,2.000000,6.000000,0.000000,0.000000,0.000000,0.000000
50%,73.500000,1.345243,5879.310247,0.000000,1.500000,153.000000,122.500000,0.000000,0.000000,0.000000,...,78.000000,381.000000,5.000000,7.000000,4.000000,8.000000,0.000000,0.000000,0.000000,0.000000
75%,108.750000,1.472059,11788.211092,0.000000,90.250000,279.500000,408.750000,0.000000,0.000000,0.000000,...,157.750000,975.750000,7.750000,9.000000,7.000000,12.000000,0.000000,0.000000,1.000000,1.000000
max,147.000000,2.000000,48029.799723,1666.000000,1258.000000,1152.000000,1642.000000,178072.000000,2697.000000,115378.000000,...,726.000000,2544.000000,35.000000,33.000000,23.000000,44.000000,1.000000,1.000000,1.000000,1.000000


In [4]:
dataset.info

<bound method DataFrame.info of      id    target  Расстояние до метро  \
0    60  1.292717          5526.692151   
1    50  1.674569         16243.183872   
2    17  1.140666          6401.230357   
3    15  1.352501          2255.498620   
4    74  1.324726          5811.209297   
..  ...       ...                  ...   
93  112       NaN         19887.712950   
94  117       NaN           485.615204   
95  126       NaN          2420.989438   
96  144       NaN         11997.377318   
97  145       NaN          2679.677085   

    Суммарное кол-во отзывов в категории "АЗС" в радиусе 150м  \
0                                                   0           
1                                                   0           
2                                                   0           
3                                                   0           
4                                                   0           
..                                                ...           
93      

Узнаем количество отсутсвующей информации во всём dataset.

In [3]:
dataset.isna().sum().sum()

29426

Теперь удалим те признаки о которых полностью ничего не известно, то есть у всех объектов этот признак не известен. Делаем это для того, чтобы немного избавиться от большого кол-ва признаков, и для облегчения обучения моделей. Также по условию, отсутсвующая информация это альтернатива нулю. В таком случае, нет смысла хранить нулевые вектор-столбцы, которые будут линейно зависимы со всеми признаками. Это может усложнить обучение моделей.  

In [7]:
dataset_dropna = dataset.dropna(axis = 1, how='all')

In [8]:
dataset_dropna

,id,target,Расстояние до метро,"Суммарное кол-во отзывов в категории ""АЗС"" в радиусе 150м","Суммарное кол-во отзывов в категории ""Алкомаркеты"" в радиусе 150м","Суммарное кол-во отзывов в категории ""Аптеки"" в радиусе 150м","Суммарное кол-во отзывов в категории ""Банки"" в радиусе 150м","Суммарное кол-во отзывов в категории ""Бизнес_центры"" в радиусе 150м","Суммарное кол-во отзывов в категории ""ВУЗы"" в радиусе 150м","Суммарное кол-во отзывов в категории ""Вокзалы"" в радиусе 150м",...,Суммарный вечерний трафик сигналов рабочих в радиусе 140м,Суммарный трафик сигналов рабочих в радиусе 140м,Суммарный утренний трафик юзеров рабочих в радиусе 140м,Суммарный дневной трафик юзеров рабочих в радиусе 140м,Суммарный вечерний трафик юзеров рабочих в радиусе 140м,Суммарный трафик юзеров рабочих в радиусе 140м,Тип БЦ,Тип хаб,Тип улица,Тип ТЦ
0,60,1.292717,5526.692151,0,274,56,156,249,0,0,...,33,187,3,6,3,6,0,0,1,0
1,50,1.674569,16243.183872,0,0,719,402,0,0,0,...,303,1302,9,12,11,15,0,0,0,1
2,17,1.140666,6401.230357,0,194,425,271,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,15,1.352501,2255.498620,0,0,0,0,1737,83,0,...,52,315,6,7,5,10,0,0,1,0
4,74,1.324726,5811.209297,0,0,71,0,0,0,0,...,257,1189,2,2,2,2,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,112,NaN,19887.712950,0,0,193,32,0,0,0,...,185,1074,6,6,7,8,0,0,0,1
94,117,NaN,485.615204,0,1258,237,103,372,60,0,...,351,1529,9,10,10,13,0,0,1,0
95,126,NaN,2420.989438,0,146,346,16,20,0,0,...,129,437,8,10,6,14,0,0,1,0
96,144,NaN,11997.377318,0,85,414,547,0,0,0,...,91,309,10,10,9,12,0,0,1,0


In [9]:
dataset_dropna.isna().sum().sum()

23938

Теперь заменим все пропуски нулями

In [10]:
dataset_dropna.fillna(0, inplace = True)

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4244: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  **kwargs


In [11]:
dataset_dropna.isna().sum().sum()

0

В исходных данных есть похожие признаки, значения которых было измерено разными метриками расстояния (heaviside, gauss). Эти признаки сильно коррелируют, ведь они похожи и увеличение одной метрики привидёт к увеличению другой и наоборот.

In [31]:
print("heaviside for 1st object: ",dataset_dropna.iloc[0]['Суммарный мобильный трафик по возрастной группе от 18 до 25 в радиусе 150м heaviside'], 
     "\ngauss for 1st object on same feature: ",dataset_dropna.iloc[0]['Суммарный мобильный трафик по возрастной группе от 18 до 25 в радиусе 150м gauss'])

heaviside for 1st object:  768910.0 
gauss for 1st object on same feature:  705112.6414124254


In [32]:
print("heaviside for 1st object: ",dataset_dropna.iloc[0]['Суммарный мобильный трафик по возрастной группе от 25 до 35 в радиусе 150м heaviside'], 
     "\ngauss for 1st object on same feature: ",dataset_dropna.iloc[0]['Суммарный мобильный трафик по возрастной группе от 25 до 35 в радиусе 150м gauss'])

heaviside for 1st object:  2955618.0 
gauss for 1st object on same feature:  2654995.075958025


In [33]:
print("heaviside for 1st object: ",dataset_dropna.iloc[0]['Суммарный мобильный трафик по доходной группе от 12000 до 46000 в радиусе 150м heaviside'], 
     "\ngauss for 1st object on same feature: ",dataset_dropna.iloc[0]['Суммарный мобильный трафик по доходной группе от 12000 до 46000 в радиусе 150м gauss'])

heaviside for 1st object:  4275490.0 
gauss for 1st object on same feature:  4003517.202764344


Как раз видим, что если увеличивается или уменьшается значение одного признака, то и уменьшается значение другого признака. Более наглядно покажем это, посчитав коэффицент корреляции между признами.

In [38]:
print("Corr coef: ", scipy.stats.linregress(dataset_dropna['Суммарный мобильный трафик по доходной группе от 12000 до 46000 в радиусе 150м heaviside'], 
      dataset_dropna['Суммарный мобильный трафик по доходной группе от 12000 до 46000 в радиусе 150м gauss']).rvalue)

Corr coef:  0.9483694233318383


Это очевидно справедливо для всех признаков измеренных метриками heaviside и gauss. Поэтому избавмися от дублирования информации.

In [19]:
gauss_columns = []
heaviside_columns = []
for i in dataset_dropna.columns:
    if 'gauss' in i:
        gauss_columns.append(i)
for i in dataset_dropna.columns:
    if 'heaviside' in i:
        heaviside_columns.append(i)
print("Len of gauss columns - ",len(gauss_columns),"\nLen of heaviside columns - ", len(heaviside_columns))

Len of gauss columns -  35 
Len of heaviside columns -  112


In [28]:
gauss_heaviside_columns = gauss_columns[:]
for i in range(len(gauss_heaviside_columns)):
    gauss_heaviside_columns[i]=gauss_heaviside_columns[i].replace('gauss', 'heaviside')
count = 0
for i in range(len(gauss_heaviside_columns)):
    if gauss_heaviside_columns[i] in heaviside_columns:
        count+=1
print("Count of identical features measured and by heaviside and by gauss - ", count)

Count of identical features measured and by heaviside and by gauss -  35


Можно сказать, что все признаки измеренные метрикой gauss, также измерены метрикой heaviside, но не все признаки измеренные метрикой heaviside, измерены метрикой gauss. В таком случае можем избавить от лишних 35 признаков, измеренных метрикой gauss.

In [29]:
dataset_dropna_dropgauss = dataset_dropna.drop(columns = gauss_columns, axis = 1)
len(dataset_dropna_dropgauss.columns)

2244

Пока что снизили кол-во признаков с 2335 до 2244 (2242, так как признаки id и target при обучение модели не используются). Дальше буду не сам отбирать признаки, а благодаря моделям, потому что в данных нельзя утверждать, что хранить выгодней: отдельно "Суммарное кол-во отзывов в категории "АЗС" в радиусе 150м", "Суммарное кол-во отзывов в категории "АЗС" в радиусе 300м", "Суммарное кол-во отзывов в категории "АЗС" в радиусе 500м"... или лучше хранить сумму этих признаков по всем категориям ("АЗС", "Алкомаркеты", "Аптеки" ...)(этот признак есть в данных) - а таких ситуации в данных очень много.

In [40]:
dataset_dropna_dropgauss

,id,target,Расстояние до метро,"Суммарное кол-во отзывов в категории ""АЗС"" в радиусе 150м","Суммарное кол-во отзывов в категории ""Алкомаркеты"" в радиусе 150м","Суммарное кол-во отзывов в категории ""Аптеки"" в радиусе 150м","Суммарное кол-во отзывов в категории ""Банки"" в радиусе 150м","Суммарное кол-во отзывов в категории ""Бизнес_центры"" в радиусе 150м","Суммарное кол-во отзывов в категории ""ВУЗы"" в радиусе 150м","Суммарное кол-во отзывов в категории ""Вокзалы"" в радиусе 150м",...,Суммарный вечерний трафик сигналов рабочих в радиусе 140м,Суммарный трафик сигналов рабочих в радиусе 140м,Суммарный утренний трафик юзеров рабочих в радиусе 140м,Суммарный дневной трафик юзеров рабочих в радиусе 140м,Суммарный вечерний трафик юзеров рабочих в радиусе 140м,Суммарный трафик юзеров рабочих в радиусе 140м,Тип БЦ,Тип хаб,Тип улица,Тип ТЦ
0,60,1.292717,5526.692151,0,274,56,156,249,0,0,...,33,187,3,6,3,6,0,0,1,0
1,50,1.674569,16243.183872,0,0,719,402,0,0,0,...,303,1302,9,12,11,15,0,0,0,1
2,17,1.140666,6401.230357,0,194,425,271,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,15,1.352501,2255.498620,0,0,0,0,1737,83,0,...,52,315,6,7,5,10,0,0,1,0
4,74,1.324726,5811.209297,0,0,71,0,0,0,0,...,257,1189,2,2,2,2,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,112,0.000000,19887.712950,0,0,193,32,0,0,0,...,185,1074,6,6,7,8,0,0,0,1
94,117,0.000000,485.615204,0,1258,237,103,372,60,0,...,351,1529,9,10,10,13,0,0,1,0
95,126,0.000000,2420.989438,0,146,346,16,20,0,0,...,129,437,8,10,6,14,0,0,1,0
96,144,0.000000,11997.377318,0,85,414,547,0,0,0,...,91,309,10,10,9,12,0,0,1,0


In [41]:
X = dataset_dropna_dropgauss.drop(["id", "target"], axis=1)
y = dataset_dropna_dropgauss['target']

В качестве train data берём первые 88 объектов, так как для последних 10 надо сделать предсказание. 

In [42]:
X_train = X[:88]
X_test = X[88:]
y_train = y[:88]

In [43]:
X_train

,Расстояние до метро,"Суммарное кол-во отзывов в категории ""АЗС"" в радиусе 150м","Суммарное кол-во отзывов в категории ""Алкомаркеты"" в радиусе 150м","Суммарное кол-во отзывов в категории ""Аптеки"" в радиусе 150м","Суммарное кол-во отзывов в категории ""Банки"" в радиусе 150м","Суммарное кол-во отзывов в категории ""Бизнес_центры"" в радиусе 150м","Суммарное кол-во отзывов в категории ""ВУЗы"" в радиусе 150м","Суммарное кол-во отзывов в категории ""Вокзалы"" в радиусе 150м","Суммарное кол-во отзывов в категории ""Детскиемагазины"" в радиусе 150м","Суммарное кол-во отзывов в категории ""Жилыедома"" в радиусе 150м",...,Суммарный вечерний трафик сигналов рабочих в радиусе 140м,Суммарный трафик сигналов рабочих в радиусе 140м,Суммарный утренний трафик юзеров рабочих в радиусе 140м,Суммарный дневной трафик юзеров рабочих в радиусе 140м,Суммарный вечерний трафик юзеров рабочих в радиусе 140м,Суммарный трафик юзеров рабочих в радиусе 140м,Тип БЦ,Тип хаб,Тип улица,Тип ТЦ
0,5526.692151,0,274,56,156,249,0,0,1268,0,...,33,187,3,6,3,6,0,0,1,0
1,16243.183872,0,0,719,402,0,0,0,3445,0,...,303,1302,9,12,11,15,0,0,0,1
2,6401.230357,0,194,425,271,0,0,0,31,0,...,0,0,0,0,0,0,0,0,1,0
3,2255.498620,0,0,0,0,1737,83,0,0,0,...,52,315,6,7,5,10,0,0,1,0
4,5811.209297,0,0,71,0,0,0,0,1051,0,...,257,1189,2,2,2,2,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,5123.453252,0,12,316,0,0,0,0,14,0,...,76,1468,6,6,6,8,0,1,0,0
84,3238.098609,0,82,477,92,18,0,0,141,0,...,87,963,6,5,6,11,0,0,1,0
85,13175.266371,0,72,170,596,0,0,0,815,0,...,200,1426,7,7,6,8,0,0,0,1
86,19176.051056,0,0,16,0,0,0,0,798,0,...,134,1528,7,8,8,11,0,0,0,1


<h1>Data Normalization with MinMaxScaler

Так как задача регрессии, будем нормировать данные

In [44]:
MinMax = MinMaxScaler()
X_train_scaled = MinMax.fit_transform(X_train)
X_test_scaled = MinMax.transform(X_test)

Должны получить значения для каждого признака (объекта) в пределе от 0 до 1

In [54]:
print(X_train_scaled[X_train_scaled<0], X_train_scaled[X_train_scaled>1+1e-10])


[] []


<h2>MAPE Realization

In [56]:
def MAPE(y_true, y_pred):
    mape = np.mean(np.abs((y_true - y_pred)/y_true))
    return mape

In [57]:
my_mape = make_scorer(MAPE, greater_is_better = True)

<h2>RandomForestRegressor with RSCV and cross_val_score

In [60]:
est = RandomForestRegressor(max_features = int(len(X_train.columns)/3))
params = {
    'n_estimators': range(100, 1000, 100),
    'max_depth' : np.linspace(5, 100, 10)
}
rgrid = RandomizedSearchCV(est, params, scoring = my_mape, cv = 5)

In [61]:
rgrid.fit(X_train_scaled, y_train)

RandomizedSearchCV(cv=5, error_score='raise-deprecating',
                   estimator=RandomForestRegressor(bootstrap=True,
                                                   criterion='mse',
                                                   max_depth=None,
                                                   max_features=747,
                                                   max_leaf_nodes=None,
                                                   min_impurity_decrease=0.0,
                                                   min_impurity_split=None,
                                                   min_samples_leaf=1,
                                                   min_samples_split=2,
                                                   min_weight_fraction_leaf=0.0,
                                                   n_estimators='warn',
                                                   n_jobs=None, oob_score=False,
                                                   random_state=...=

In [62]:
rgrid_best = rgrid.best_estimator_
rgrid_best

RandomForestRegressor(bootstrap=True, criterion='mse',
                      max_depth=57.77777777777778, max_features=747,
                      max_leaf_nodes=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=300, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [90]:
rfr_feature_importances = []
for i, k in enumerate(rgrid_best.feature_importances_):
    if abs(k)>5e-03:
        print("Coef for ", X_train.columns[i],": ", k)
        rfr_feature_importances.append(X_train.columns[i])

Coef for  Суммарное кол-во отзывов в категории "Косметика" в радиусе 150м :  0.0068803876056866475
Coef for  Суммарное кол-во отзывов в категории "Хозтоварыидом" в радиусе 150м :  0.0261471019347219
Coef for  Суммарное кол-во отзывов в категории "ТЦ" в радиусе 300м :  0.008176081228110978
Coef for  Суммарное кол-во отзывов в категории "ТЦ" в радиусе 500м :  0.0050987335499683385
Coef for  Суммарное кол-во отзывов в категории "Электроникаибытоваятехника" в радиусе 500м :  0.005767357631843984
Coef for  Среднее кол-во отзывов в категории "Косметика" в радиусе 150м :  0.006687576282262772
Coef for  Среднее кол-во отзывов в категории "ТЦ" в радиусе 300м :  0.011487783103383891
Coef for  Среднее кол-во отзывов в категории "ТЦ" в радиусе 500м :  0.011765345289616936
Coef for  Суммарный средний рейтинг объектов в категории "Одеждаиобувь" в радиусе 300м :  0.007003712357644397
Coef for  Средний средний рейтинг объектов в категории "Электроникаибытоваятехника" в радиусе 300м :  0.00767056267578

In [91]:
len(rfr_feature_importances)

29

In [63]:
cross_score_rfr = cross_val_score(rgrid_best, X_train_scaled, y_train, scoring = my_mape, cv = 5).mean()

In [65]:
cross_score_rfr

0.09582424319635886

In [64]:
predict_rfr = rgrid_best.predict(X_test_scaled)

In [66]:
predict_rfr

array([1.56799795, 1.24293159, 1.47184698, 1.44984133, 1.34135446,
       1.42740691, 1.39774898, 1.43622523, 1.40848709, 1.36052752])

<h1> XGBoost

In [92]:
pip install xgboost

Note: you may need to restart the kernel to use updated packages.


In [93]:
import xgboost as xgb

In [94]:
data_dmatrix = xgb.DMatrix(data = X_train_scaled, label = y_train)

In [96]:
xgb_param = {
    'alpha':np.linspace(0.01, 10, 10),
    'n_estimators' : range(10,1000, 200)
}

In [97]:
xgbgrid = GridSearchCV(xgb.XGBRegressor(), xgb_param, cv = 5, scoring = my_mape)

In [98]:
xgbgrid.fit(X_train_scaled, y_train)

/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=False, eval_metric=None,
                                    gamma=None, gpu_id=None, grow_policy=None,
                                    importance_type=None,
                                    interaction_constraints=None,
                                    learnin...
                                    monotone_constraints=None, n_estimators=100,
                                    n_jobs=None, num_parallel_tree=None,
                                    objective='reg:squarederror',
                                    predictor=None, random_state=None,
  

In [99]:
xgb_best = xgbgrid.best_estimator_

In [100]:
xgb_best

XGBRegressor(alpha=10.0, base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=10, n_jobs=0,
             num_parallel_tree=1, objective='reg:squarederror',
             predictor='auto', random_state=0, ...)

In [101]:
cross_score_xgb = cross_val_score(xgb_best, X_train_scaled, y_train, cv = 5, scoring = my_mape).mean()

In [102]:
cross_score_xgb

0.13156376945025589

In [103]:
predict_xgb = xgb_best.predict(X_test_scaled)

In [104]:
predict_xgb

array([1.2616302, 1.2616302, 1.2616302, 1.2616302, 1.03186  , 1.2616302,
       1.1887321, 1.2282051, 1.032    , 1.2616302], dtype=float32)

In [113]:
xgb_feature_importances = []
for i, k in enumerate(xgb_best.feature_importances_):
    if abs(k)>5e-03:
        print("Coef for '",X_train.columns[i],"': ", k)
        xgb_feature_importances.append(X_train.columns[i])

Coef for ' Суммарное кол-во отзывов в категории "Общепит" в радиусе 150м ':  0.016943278
Coef for ' Суммарное кол-во отзывов в категории "Пекарни" в радиусе 150м ':  0.01368127
Coef for ' Суммарное кол-во отзывов в категории "Сетевыепродукты" в радиусе 150м ':  0.041409653
Coef for ' Суммарное кол-во отзывов в категории "Цветы" в радиусе 150м ':  0.01322361
Coef for ' Суммарное кол-во отзывов в категории "Аптеки" в радиусе 300м ':  0.019218931
Coef for ' Суммарное кол-во отзывов в категории "Стадионы" в радиусе 300м ':  0.054773413
Coef for ' Суммарное кол-во отзывов в категории "Банки" в радиусе 500м ':  0.009934249
Coef for ' Среднее кол-во отзывов в категории "ВУЗы" в радиусе 150м ':  0.2004858
Coef for ' Среднее кол-во отзывов в категории "Косметика" в радиусе 150м ':  0.19363548
Coef for ' Среднее кол-во отзывов в категории "Цветы" в радиусе 150м ':  0.05169477
Coef for ' Среднее кол-во отзывов в категории "АЗС" в радиусе 300м ':  0.009388888
Coef for ' Среднее кол-во отзывов в ка

In [114]:
count = 0
for i in range(len(xgb_feature_importances)):
    if xgb_feature_importances[i] in rfr_feature_importances:
        count+=1
        print("Feature '", xgb_feature_importances[i], "' is important for rfr and for xgb ")
print("Count of same imporant features after rfr training and XGBoost training: ", count)

Feature ' Среднее кол-во отзывов в категории "Косметика" в радиусе 150м ' is important for rfr and for xgb 
Count of same imporant features after rfr training and XGBoost training:  1


На данный момент имеем два массива признаков с относительно большими коэффицентами, получившимися после обучения двух композиций: RandomForest и XGBoost. Причём, можно отметить, что уже сейчас есть модель с хорошей точностью на валидации (rfr). Сейчас попробуем проинтерпритировать признаки, которые модели считают "важными". Начнём с rfr:

In [115]:
rfr_feature_importances

['Суммарное кол-во отзывов в категории "Косметика" в радиусе 150м',
 'Суммарное кол-во отзывов в категории "Хозтоварыидом" в радиусе 150м',
 'Суммарное кол-во отзывов в категории "ТЦ" в радиусе 300м',
 'Суммарное кол-во отзывов в категории "ТЦ" в радиусе 500м',
 'Суммарное кол-во отзывов в категории "Электроникаибытоваятехника" в радиусе 500м',
 'Среднее кол-во отзывов в категории "Косметика" в радиусе 150м',
 'Среднее кол-во отзывов в категории "ТЦ" в радиусе 300м',
 'Среднее кол-во отзывов в категории "ТЦ" в радиусе 500м',
 'Суммарный средний рейтинг объектов в категории "Одеждаиобувь" в радиусе 300м',
 'Средний средний рейтинг объектов в категории "Электроникаибытоваятехника" в радиусе 300м',
 'Средний средний рейтинг объектов в категории "Электроникаибытоваятехника" в радиусе 500м',
 'Суммарный вечерний пешеходный трафик сигналов в радиусе 300м',
 'Суммарный вечерний пешеходный трафик сигналов в радиусе 700м',
 'Суммарный вечерний пешеходный трафик сигналов в радиусе 1000м',
 'Макс

Наиболее логичные признаки:

   1. Суммарный/максимальны/средний утренний/вечерний автомобильный/пешеходный трафик (так как логично предположить что в кофейню заходят чаще утром (перед работой, учёбой, чтобы бодрее себя чувстовать) и вечером (чтобы отдохнуть после рабочего дня). Сюда же относятся признаки:
    - "Суммарный утренний пешеходный трафик"
    - "Средний  пешеходный трафик"
    - "Средний утренний автомобильный трафик"
    - "Суммарный утренний автомобильный трафик"
    - "Суммарный  автомобильный трафик"
    
   Также можно отметить что признак "Максимальный дневной пешеходный трафик" тоже имеет значение, так как и днём во время ланча, часто заходят в кофейни.
    
   2. Признаки связанный с кол-вом отзывов:
       - "Суммарное кол-во отзывов в категории "ТЦ" в радиусе 300"
       - "Суммарное кол-во отзывов в категории "ТЦ" в радиусе 500"
       - "Среднее кол-во отзывов в категории "ТЦ" в радиусе 300м"
       - "Среднее кол-во отзывов в категории "ТЦ" в радиусе 500м"
   
       -- наиболее логичные,так как отзывы именно этой категории больше всего вляют на выбор кофейни (много отзывов, и много читающих, так как довольно популярная категория)

In [116]:
xgb_feature_importances

['Суммарное кол-во отзывов в категории "Общепит" в радиусе 150м',
 'Суммарное кол-во отзывов в категории "Пекарни" в радиусе 150м',
 'Суммарное кол-во отзывов в категории "Сетевыепродукты" в радиусе 150м',
 'Суммарное кол-во отзывов в категории "Цветы" в радиусе 150м',
 'Суммарное кол-во отзывов в категории "Аптеки" в радиусе 300м',
 'Суммарное кол-во отзывов в категории "Стадионы" в радиусе 300м',
 'Суммарное кол-во отзывов в категории "Банки" в радиусе 500м',
 'Среднее кол-во отзывов в категории "ВУЗы" в радиусе 150м',
 'Среднее кол-во отзывов в категории "Косметика" в радиусе 150м',
 'Среднее кол-во отзывов в категории "Цветы" в радиусе 150м',
 'Среднее кол-во отзывов в категории "АЗС" в радиусе 300м',
 'Среднее кол-во отзывов в категории "Электроникаибытоваятехника" в радиусе 500м',
 'Среднее кол-во отзывов в категории "Косметика" в радиусе 700м',
 'Суммарный средний рейтинг объектов в категории "Пекарни" в радиусе 150м',
 'Суммарный средний рейтинг объектов в категории "Продукты" 

Наиболее логичные признаки:

   1. Признаки связанный с кол-вом отзывов:
       - "Суммарное кол-во отзывов в категории "Общепит" в радиусе 150м"
       - "Суммарное кол-во отзывов в категории "Пекарни" в радиусе 150м"
       - "Среднее кол-во отзывов в категории "ВУЗы" в радиусе 150м"
       - "Суммарный средний рейтинг объектов в категории "Пекарни" в радиусе 150м"
   
       Так как отзывы этих категорий логично связаны с товарооборотом кофейни, ведь, например, опираясь на 3-ий признак, студенты часто во время свободных пар, проводят время в кофейнях, и можно сказать, что студенты являются хорошей прибылью, что влечёт за собой прирост товарооборота. 1-ий, 2-ий и 4-ий признаки можно объеденить в одно и сказать, что, чем больше будет отзывов в этих категориях, которые связаны с идеей кофейни, тем выше вероятность прироста прибыли, товарооборота. 
       
       
   2. Оставшиеся признаки связанные с трафиком. Модель делает упор на признаках связанные с утренними замерами. Ни одного вечернего замера.     

Правильнее будет считать более логичными признаками, которые выявлены после обучения RFR, так как больше имеется логических связей. Попробуем их использовать для обучения новых моделей.

In [119]:
X_new = X[rfr_feature_importances]

In [120]:
X_new

,"Суммарное кол-во отзывов в категории ""Косметика"" в радиусе 150м","Суммарное кол-во отзывов в категории ""Хозтоварыидом"" в радиусе 150м","Суммарное кол-во отзывов в категории ""ТЦ"" в радиусе 300м","Суммарное кол-во отзывов в категории ""ТЦ"" в радиусе 500м","Суммарное кол-во отзывов в категории ""Электроникаибытоваятехника"" в радиусе 500м","Среднее кол-во отзывов в категории ""Косметика"" в радиусе 150м","Среднее кол-во отзывов в категории ""ТЦ"" в радиусе 300м","Среднее кол-во отзывов в категории ""ТЦ"" в радиусе 500м","Суммарный средний рейтинг объектов в категории ""Одеждаиобувь"" в радиусе 300м","Средний средний рейтинг объектов в категории ""Электроникаибытоваятехника"" в радиусе 300м",...,Суммарный утренний автомобильный трафик сигналов резидентов в радиусе 500м,Суммарный автомобильный трафик сигналов резидентов в радиусе 500м,Максимальный дневной пешеходный трафик сигналов резидентов в радиусе 500м,Максимальный вечерний пешеходный трафик сигналов резидентов в радиусе 500м,Максимальный пешеходный трафик сигналов резидентов в радиусе 500м,Суммарный вечерний пешеходный трафик сигналов рабочих в радиусе 500м,Средний утренний автомобильный трафик сигналов рабочих в радиусе 500м,Суммарный вечерний трафик сигналов рабочих в радиусе 500м,Суммарный утренний пешеходный трафик сигналов в радиусе 140м,Средний пешеходный трафик юзеров в радиусе 140м
0,10,208,14504,14504,26795,10.000000,4834.666667,4834.666667,128.4,4.090909,...,675,3440,378,215,1007,2226,52.043478,3134,1432,1104
1,1198,3139,232203,232203,5806,47.920000,38700.500000,38700.500000,631.9,4.066667,...,4668,20407,3316,4266,15900,5935,84.306122,8430,3895,4821
2,166,159,0,0,183,83.000000,0.000000,0.000000,145.3,4.800000,...,562,2327,365,330,1046,1713,50.510204,2464,347,234
3,0,0,0,0,54,0.000000,0.000000,0.000000,25.8,4.400000,...,1548,15617,1012,846,3479,2822,63.528302,7855,445,844
4,9,3237,24174,24174,600,9.000000,24174.000000,24174.000000,165.4,4.275000,...,2427,12891,1406,1346,5496,5794,59.313725,8549,819,1014
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,284,126,34482,34482,660,31.555556,34482.000000,34482.000000,235.2,3.930000,...,108,666,250,107,872,802,35.266667,1053,2272,2169
94,373,0,0,0,545,186.500000,0.000000,0.000000,21.7,4.100000,...,1339,4548,1199,994,3967,3080,72.360000,5152,2702,2280
95,263,45,0,0,2306,87.666667,0.000000,0.000000,47.1,4.566667,...,2900,14042,2504,1950,9242,6037,89.409091,9578,1179,1620
96,224,7,6128,6128,337,224.000000,2042.666667,2042.666667,71.8,3.771429,...,2876,14847,1242,2284,6114,6103,47.050847,9718,2528,2080


In [121]:
X_new_train = X_new[:88]
X_new_test = X_new[88:]
y_new_train = y_train

In [122]:
X_new_train_scaled = MinMax.fit_transform(X_new_train)
X_new_test_scaled = MinMax.transform(X_new_test)

<h1>RFR on new train data

In [123]:
rfr_new = RandomForestRegressor(max_features = int(len(X_new_train.columns)/3), n_jobs = -1)
params_rfr = {
    'n_estimators': range(100, 1000, 100),
    'max_depth' : np.linspace(5, 100, 10)
}
rfr_new_grid = GridSearchCV(rfr_new, params_rfr, scoring = my_mape, cv = 5)
rfr_new_grid.fit(X_new_train_scaled, y_new_train)
rfr_new_grid_best = rfr_new_grid.best_estimator_
rfr_new_grid_best

RandomForestRegressor(bootstrap=True, criterion='mse',
                      max_depth=89.44444444444444, max_features=9,
                      max_leaf_nodes=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=-1, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [171]:
cross_score_rfr_new = cross_val_score(rfr_new_grid_best, X_new_train_scaled, y_new_train, cv = 5, scoring = my_mape).mean()

In [172]:
cross_score_rfr_new

0.08039358353296473

In [126]:
predict_rfr_new = rfr_new_grid_best.predict(X_new_test_scaled)

In [128]:
predict_rfr_new 

array([1.64754931, 1.2800803 , 1.49516837, 1.4526501 , 1.35953101,
       1.40121588, 1.34285521, 1.48028079, 1.35454605, 1.36787177])

Точность получили ещё выше. Теперь попробуем новые модели

<h1>SGDRegressor

In [135]:
sgdr = SGDRegressor(penalty = 'l2')
params_sgdr = {
    'alpha': np.linspace(0.0001, 10, 15)
}
sgdr_grid = GridSearchCV(sgdr, params_sgdr, cv=5, scoring = my_mape)

In [136]:
sgdr_grid.fit(X_new_train_scaled, y_new_train)

/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=SGDRegressor(alpha=0.0001, average=False,
                                    early_stopping=False, epsilon=0.1,
                                    eta0=0.01, fit_intercept=True,
                                    l1_ratio=0.15, learning_rate='invscaling',
                                    loss='squared_loss', max_iter=1000,
                                    n_iter_no_change=5, penalty='l2',
                                    power_t=0.25, random_state=None,
                                    shuffle=True, tol=0.001,
                                    validation_fraction=0.1, ver...
             param_grid={'alpha': array([1.00000000e-04, 7.14378571e-01, 1.42865714e+00, 2.14293571e+00,
       2.85721429e+00, 3.57149286e+00, 4.28577143e+00, 5.00005000e+00,
       5.71432857e+00, 6.42860714e+00, 7.14288571e+00, 7.85716429e+00,
       8.57144286e+00, 9.28572143e+00, 1.00000000e+01])},
             pre_disp

In [137]:
sgdr_grid_best = sgdr_grid.best_estimator_

In [138]:
sgdr_grid_best

SGDRegressor(alpha=0.0001, average=False, early_stopping=False, epsilon=0.1,
             eta0=0.01, fit_intercept=True, l1_ratio=0.15,
             learning_rate='invscaling', loss='squared_loss', max_iter=1000,
             n_iter_no_change=5, penalty='l2', power_t=0.25, random_state=None,
             shuffle=True, tol=0.001, validation_fraction=0.1, verbose=0,
             warm_start=False)

In [162]:
cross_score_sgd = cross_val_score(sgdr_grid_best, X_new_train_scaled, y_new_train, cv = 5, scoring = my_mape).mean()

In [163]:
cross_score_sgd

0.14606323603768465

Ошибка довольно большая.

<h1>Ridge

In [144]:
ridge = Ridge()
params_ridge = {
    'alpha': np.linspace(0.0001, 10, 20),
    'solver': ["auto", "svd", "cholesky", "lsqr", "sparse_cg", "sag", "saga"]
}
ridge_grid = GridSearchCV(ridge, params_ridge, cv=5, scoring = my_mape)
ridge_grid.fit(X_new_train_scaled, y_new_train)
ridge_grid_best = ridge_grid.best_estimator_
ridge_grid_best

/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


Ridge(alpha=0.0001, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=None, solver='sparse_cg', tol=0.001)

In [164]:
cross_score_ridge = cross_val_score(ridge_grid_best, X_new_train_scaled, y_new_train, cv = 5, scoring = my_mape).mean()

In [165]:
cross_score_ridge

0.1231438595767903

<h1>AdaBoostRegressor on ridge estimator

In [150]:
abr = AdaBoostRegressor(base_estimator = ridge_grid_best)
params_abr = {
    'n_estimators': range(50, 1000, 75),
    'learning_rate': np.linspace(0.001, 5, 20)
}
abr_grid = GridSearchCV(abr, params_abr,cv = 5, scoring = my_mape)
abr_grid.fit(X_new_train_scaled, y_new_train)

/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=AdaBoostRegressor(base_estimator=Ridge(alpha=0.0001,
                                                              copy_X=True,
                                                              fit_intercept=True,
                                                              max_iter=None,
                                                              normalize=False,
                                                              random_state=None,
                                                              solver='sparse_cg',
                                                              tol=0.001),
                                         learning_rate=1.0, loss='linear',
                                         n_estimators=50, random_state=None),
             iid='warn', n_jobs=None,
             param_grid={'learning_rate': array([1.00...
       1.05342105e+00, 1.31652632e+00, 1.57963158e+00, 1.84273684e+00,
   

In [151]:
abr_grid_best = abr_grid.best_estimator_
abr_grid_best

AdaBoostRegressor(base_estimator=Ridge(alpha=0.0001, copy_X=True,
                                       fit_intercept=True, max_iter=None,
                                       normalize=False, random_state=None,
                                       solver='sparse_cg', tol=0.001),
                  learning_rate=4.473789473684211, loss='linear',
                  n_estimators=950, random_state=None)

In [161]:
cross_score_abr = cross_val_score(abr_grid_best, X_new_train_scaled, y_new_train, cv = 5, scoring = my_mape).mean()

Градиентный бустинг улучшил результат, но всё равно до сих пор самая лучшая модель это RandomForestRegressor на усечённых данных

In [173]:
print("Mean cross validation score:")
print("RandomForstRegressor on full data:", cross_score_rfr)
print("XGBoostRegressor on full data:", cross_score_xgb)
print("RandomForstRegressor on data with important features:", cross_score_rfr_new)
print("SGDRegressor on data with important features", cross_score_sgd)
print("Ridge on data with important features", cross_score_ridge)
print("AdaBoostRegressor on data with important features", cross_score_abr)

Mean cross validation score:
RandomForstRegressor on full data: 0.09582424319635886
XGBoostRegressor on full data: 0.13156376945025589
RandomForstRegressor on data with important features: 0.08039358353296473
SGDRegressor on data with important features 0.14606323603768465
Ridge on data with important features 0.1231438595767903
AdaBoostRegressor on data with important features 0.12257020605630715


In [174]:
predict_rfr_new

array([1.64754931, 1.2800803 , 1.49516837, 1.4526501 , 1.35953101,
       1.40121588, 1.34285521, 1.48028079, 1.35454605, 1.36787177])

In [178]:
data_answer = pd.read_excel("форма для заполнения.xlsx")

In [179]:
data_answer

,id,target predicted
0,3,NaN
1,25,NaN
2,55,NaN
3,69,NaN
4,109,NaN
5,112,NaN
6,117,NaN
7,126,NaN
8,144,NaN
9,145,NaN


In [180]:
data_answer['target predicted'] = predict_rfr_new

In [181]:
data_answer

,id,target predicted
0,3,1.647549
1,25,1.280080
2,55,1.495168
3,69,1.452650
4,109,1.359531
5,112,1.401216
6,117,1.342855
7,126,1.480281
8,144,1.354546
9,145,1.367872


In [182]:
data_answer.to_excel("answer_cofe.xlsx")